In [1]:
# ruff: noqa
import jax

jax.config.update("jax_enable_x64", True)

import astropy.units as u
import jax.numpy as jnp
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroquery.jplhorizons import Horizons

from jorbit import Ephemeris, Observations, Particle
from jorbit.accelerations import (
    create_default_ephemeris_acceleration_func,
    create_static_default_acceleration_func,
    create_static_default_on_sky_acc_func,
)
from jorbit.accelerations.static_helpers import (
    generate_perturber_chebyshev_coeffs,
    get_all_dynamic_intermediate_dts,
    precompute_perturber_positions,
)
from jorbit.astrometry.sky_projection import on_sky, tangent_plane_projection
from jorbit.integrators import ias15_static_evolve, initialize_ias15_integrator_state


In [2]:
nights = [Time("2025-01-01 07:00"), Time("2026-01-02 07:00"), Time("2027-01-05 07:00")]

times = []
for n in nights:
    times.extend([n + i * 1 * u.hour for i in range(3)])
times = Time(times)

obj = Horizons(id="274301", location="695@399", epochs=times.utc.jd)
pts = obj.ephemerides(extra_precision=True, quantities="1")

coords = SkyCoord(pts["RA"], pts["DEC"], unit=(u.deg, u.deg))
times = Time(pts["datetime_jd"], format="jd", scale="utc")

obs = Observations(
    observed_coordinates=coords,
    times=times,
    observatories="kitt peak",
    astrometric_uncertainties=1 * u.arcsec,
)

p = Particle.from_horizons("274301", Time("2024-01-01"))
p = Particle.from_horizons(
    "274301", Time("2026-01-01"), observations=obs, fit_seed=p.keplerian_state
)

In [4]:
p.static_residuals(p.keplerian_state)

Array([[ 1.53428363e-08, -1.10269611e-06],
       [ 1.15308068e-06, -1.90992495e-06],
       [-2.19604413e-07, -1.69558822e-06],
       [ 8.36264998e-07,  8.84169485e-07],
       [ 4.95231516e-07, -1.02278909e-06],
       [-1.87677561e-07, -9.59756353e-07],
       [-1.83491679e-06,  1.09244731e-06],
       [-9.10352970e-07, -1.38669385e-06],
       [-1.02579199e-06,  8.27720207e-08]], dtype=float64)